In [69]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback

chat = ChatOpenAI(
    temperature = 0.1,
    #streaming = True,
    #callbacks =[
    #    StreamingStdOutCallbackHandler(),
    #]
)

with get_openai_callback() as usage:
    chat.predict("What is the recipe for soju")
    print(usage)


[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: What is the recipe for soju"
  ]
}
[llm/end] [1:llm:ChatOpenAI] [19.82s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Here is a simple recipe for making soju at home:\n\nIngredients:\n- 1 cup of rice\n- 1 cup of nuruk (Korean fermentation starter)\n- 10 cups of water\n- 1 tablespoon of yeast\n- 1 cup of sugar (optional, for sweetening)\n\nInstructions:\n1. Rinse the rice thoroughly and soak it in water for about 1 hour.\n2. Drain the soaked rice and transfer it to a large pot. Add 10 cups of water and bring it to a boil.\n3. Reduce the heat to low and simmer the rice for about 30 minutes, or until it becomes soft and mushy.\n4. Remove the pot from heat and let it cool down to room temperature.\n5. Once the rice has cooled, transfer it to a large fermentation container or jar.\n6. Add the nuruk and yeast to the container and mix well with the rice.\n7. Cover the c

In [68]:
chat.predict("How do you make italian pasta")

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: How do you make italian pasta"
  ]
}
[llm/end] [1:llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nHere's a step-by-step guide to making Italian pasta:\n\n1. On a clean surface or in a large mixing bowl, pour the flour and create a well in the center.\n2. Crack the eggs into the well and add the salt.\n3. Using a fork or your fingers, gradually mix the eggs into the flour, incorporating a little bit at a time.\n4. Once the dough starts to come together, knead it with your hands until it forms a smooth and elastic ball. If the dough is too dry, you can add a little water, one tablespoon at a time, until it reaches the desired consistency.\n5. Once the dough is formed, cover it with a clean k

"To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nHere's a step-by-step guide to making Italian pasta:\n\n1. On a clean surface or in a large mixing bowl, pour the flour and create a well in the center.\n2. Crack the eggs into the well and add the salt.\n3. Using a fork or your fingers, gradually mix the eggs into the flour, incorporating a little bit at a time.\n4. Once the dough starts to come together, knead it with your hands until it forms a smooth and elastic ball. If the dough is too dry, you can add a little water, one tablespoon at a time, until it reaches the desired consistency.\n5. Once the dough is formed, cover it with a clean kitchen towel or plastic wrap and let it rest for about 30 minutes. This will allow the gluten to relax and make the dough easier to work with.\n6. After resting, divide the dough into smaller portions. You can use a pasta machine or a rol